In [1]:
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
from time import time  # To time our operations
import pandas as pd
from nltk.tokenize import RegexpTokenizer

C:\Users\alber\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [4]:
pd.set_option('max_colwidth', 1000)
pd.options.display.max_rows = 500
# Read file
df = pd.read_pickle('df_orig.pkl')

In [5]:
print(df.shape)
df.head()

(66806, 2)


,Doc_Id,consumer_complaint_narrative
190126,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
190135,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
190155,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
190207,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
190208,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [6]:
df=df.reset_index(drop=True)

In [7]:
df.head()

,Doc_Id,consumer_complaint_narrative
0,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
1,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
2,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
3,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
4,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [13]:
# df_clean=df[['consumer_complaint_narrative']]
df['clean'] = df['consumer_complaint_narrative'].str.replace('X', '')
df['clean'] = df['clean'].str.replace('//', '')

df = df.dropna().drop_duplicates()
df.shape

(66806, 3)

In [14]:
df_clean= df[['clean']]
df_clean.head()

,clean
0,has claimed I owe them {$27.00} for years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on 2015, th..."
2,"In my wages that I earned at my job decreased by almost half, by I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called . They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork began to be processed for some kind of ..."
3,"I have an open and current mortgage with Chase Bank # . Chase is reporting the loan payments to but is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter BK discharged dated 2013. Chase keeps referring to BK Law for Chapter and we keep providing documentation for Chapter , and the account should be open and current with all the payments \n"
4,"was submitted . At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in . I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they are going to remove it from all my b..."


In [15]:
# grab posts
sentences = df_clean.clean.astype('str').tolist()

In [16]:
sentences[1]

'Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in  2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on   2015, there

In [17]:
# tokenize
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in sentences]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [18]:
sentences_tokenized[1]

['due',
 'to',
 'inconsistencies',
 'in',
 'the',
 'amount',
 'owed',
 'that',
 'i',
 'was',
 'told',
 'by',
 'm',
 't',
 'bank',
 'and',
 'the',
 'amount',
 'that',
 'was',
 'reported',
 'to',
 'the',
 'credit',
 'reporting',
 'agencies',
 'i',
 'was',
 'advised',
 'to',
 'write',
 'a',
 'good',
 'will',
 'letter',
 'in',
 'order',
 'to',
 'address',
 'the',
 'issue',
 'and',
 'request',
 'the',
 'negative',
 'entry',
 'be',
 'removed',
 'from',
 'my',
 'credit',
 'report',
 'all',
 'together',
 'i',
 'had',
 'a',
 'vehicle',
 'that',
 'was',
 'stolen',
 'and',
 'it',
 'was',
 'declared',
 'a',
 'total',
 'loss',
 'by',
 'insurance',
 'company',
 'the',
 'insurance',
 'company',
 'and',
 'the',
 'gap',
 'insurancw',
 'companypaid',
 'the',
 'outstanding',
 'balance',
 'of',
 'the',
 'loan',
 'but',
 'i',
 'was',
 'told',
 'by',
 'm',
 't',
 'bank',
 'that',
 'there',
 'was',
 'still',
 'a',
 'balance',
 'due',
 'on',
 'the',
 'loan',
 'in',
 'good',
 'faith',
 'without',
 'having',
 '

# Fine Tuning a Word2vec Model

In [19]:
# update existing embedding w/ kaggle data
model_2 = Word2Vec(size=300, min_count=1)
model_2.build_vocab(sentences_tokenized)
total_examples = model_2.corpus_count

INFO - 03:24:52: collecting all words and their counts
INFO - 03:24:52: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 03:24:53: PROGRESS: at sentence #10000, processed 1814001 words, keeping 20810 word types
INFO - 03:24:53: PROGRESS: at sentence #20000, processed 3578866 words, keeping 29022 word types
INFO - 03:24:53: PROGRESS: at sentence #30000, processed 5368879 words, keeping 35682 word types
INFO - 03:24:54: PROGRESS: at sentence #40000, processed 7188444 words, keeping 41618 word types
INFO - 03:24:54: PROGRESS: at sentence #50000, processed 9005637 words, keeping 46531 word types
INFO - 03:24:54: PROGRESS: at sentence #60000, processed 10792194 words, keeping 50154 word types
INFO - 03:24:55: collected 52463 word types from a corpus of 12055320 raw words and 66806 sentences
INFO - 03:24:55: Loading a fresh vocabulary
INFO - 03:24:55: effective_min_count=1 retains 52463 unique words (100% of original 52463, drops 0)
INFO - 03:24:55: effective_min_coun

In [20]:
total_examples

66806

In [21]:
model_2.intersect_word2vec_format("C:/Users/alber/Downloads/GoogleNews-vectors-negative300.bin", binary=True, lockf=1.0)
model_2.train(sentences_tokenized, total_examples=total_examples, epochs=5)

INFO - 03:25:09: loading projection weights from C:/Users/alber/Downloads/GoogleNews-vectors-negative300.bin
INFO - 03:25:45: merged 28538 vectors into (52463, 300) matrix from C:/Users/alber/Downloads/GoogleNews-vectors-negative300.bin
INFO - 03:25:45: training model with 3 workers on 52463 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 03:25:46: EPOCH 1 - PROGRESS: at 6.65% examples, 581348 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:47: EPOCH 1 - PROGRESS: at 13.58% examples, 588660 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:48: EPOCH 1 - PROGRESS: at 20.81% examples, 591316 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:49: EPOCH 1 - PROGRESS: at 27.98% examples, 592522 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:50: EPOCH 1 - PROGRESS: at 35.26% examples, 592817 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:51: EPOCH 1 - PROGRESS: at 42.10% examples, 591957 words/s, in_qsize 5, out_qsize 0
INFO - 03:25:52: EPOCH 1 - PROGRESS: at 49.01% e

(43143256, 60276600)

In [22]:
# gensim flavored word2vec model (smaller)
model_2.save("cfpb_word2vec_gensim.model")

# generic word2vec model
model_2.wv.save_word2vec_format("cfpb_word2vec.model")

INFO - 03:27:06: saving Word2Vec object under cfpb_word2vec_gensim.model, separately None
INFO - 03:27:06: storing np array 'vectors' to cfpb_word2vec_gensim.model.wv.vectors.npy
INFO - 03:27:07: not storing attribute vectors_norm
INFO - 03:27:07: storing np array 'syn1neg' to cfpb_word2vec_gensim.model.trainables.syn1neg.npy
INFO - 03:27:07: not storing attribute cum_table
INFO - 03:27:07: saved cfpb_word2vec_gensim.model
INFO - 03:27:07: storing 52463x300 projection weights into cfpb_word2vec.model


In [23]:
model_2.wv.most_similar(positive=["foreclosure"],topn=20)

INFO - 03:27:25: precomputing L2-norms of word weight vectors
C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('forclosure', 0.7982729077339172),
 ('foreclosures', 0.5736782550811768),
 ('foreclosing', 0.551282525062561),
 ('foreclose', 0.5356031656265259),
 ('default', 0.5129393935203552),
 ('eviction', 0.5032855868339539),
 ('litigation', 0.49408984184265137),
 ('foreclosed', 0.480776309967041),
 ('sale', 0.47467759251594543),
 ('sheriff', 0.4598076343536377),
 ('evictions', 0.4582887291908264),
 ('beganforeclosure', 0.43819740414619446),
 ('shortsale', 0.43249696493148804),
 ('trustee', 0.43070441484451294),
 ('modification', 0.42758411169052124),
 ('motion', 0.4255492091178894),
 ('forclose', 0.4247853755950928),
 ('auction', 0.41876736283302307),
 ('forecloser', 0.4185132384300232),
 ('preforeclosure', 0.41293734312057495)]

In [24]:
model_2.wv.most_similar(positive=["mod"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('modification', 0.8594411015510559),
 ('remodification', 0.6719957590103149),
 ('modifications', 0.6642035841941833),
 ('modifcation', 0.6107051372528076),
 ('hamp', 0.5910806059837341),
 ('modified', 0.5837326049804688),
 ('harp', 0.5516812801361084),
 ('modifying', 0.5238307118415833),
 ('forbearance', 0.5112001895904541),
 ('usda', 0.5107383728027344),
 ('modify', 0.49947190284729004),
 ('rma', 0.49884259700775146),
 ('mods', 0.49727949500083923),
 ('application', 0.4930369257926941),
 ('conventional', 0.4770098328590393),
 ('hafa', 0.4716576933860779),
 ('deferment', 0.469094842672348),
 ('helpqualify', 0.4599950611591339),
 ('fha', 0.4582020342350006),
 ('recast', 0.45801037549972534)]

In [25]:
model_2.wv.most_similar(positive=["delinquent"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('late', 0.6248523592948914),
 ('deliquent', 0.5992037653923035),
 ('default', 0.5291082859039307),
 ('delinquency', 0.5227220058441162),
 ('overdue', 0.5220245122909546),
 ('behind', 0.48745405673980713),
 ('arrears', 0.4582290053367615),
 ('past', 0.43209806084632874),
 ('unpaid', 0.4244455397129059),
 ('defaulted', 0.41280093789100647),
 ('negative', 0.4043262302875519),
 ('current', 0.39643949270248413),
 ('inaccurately', 0.3957720398902893),
 ('missed', 0.3933451771736145),
 ('delinquencies', 0.3923983573913574),
 ('chargeoff', 0.38231396675109863),
 ('nonpayment', 0.38187724351882935),
 ('overdrawn', 0.3806421756744385),
 ('closed', 0.3701050877571106),
 ('inactive', 0.3670114576816559)]

In [26]:
model_2.wv.most_similar(positive=["disputes"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dispute', 0.6710425615310669),
 ('complaints', 0.6335346698760986),
 ('requests', 0.5534314513206482),
 ('investigations', 0.5092790722846985),
 ('inaccuracies', 0.4727935194969177),
 ('complaint', 0.47060421109199524),
 ('compliant', 0.4435334801673889),
 ('disagreements', 0.4419194757938385),
 ('discussions', 0.44064366817474365),
 ('discrepancies', 0.4365085959434509),
 ('inquiries', 0.43578869104385376),
 ('lawsuits', 0.4332212805747986),
 ('grievances', 0.4309554696083069),
 ('correspondences', 0.4277231991291046),
 ('disput', 0.41459810733795166),
 ('disputed', 0.4087992012500763),
 ('inquires', 0.4034891128540039),
 ('correspondence', 0.4019061326980591),
 ('communications', 0.40144070982933044),
 ('entries', 0.4009189009666443)]

In [27]:
len(model_2.wv.vocab)

52463

In [28]:
print(model_2.wv.vectors)

[[ 1.5232882e-01 -5.4447430e-01 -1.7120501e-01 ... -1.0316514e+00
   1.0673243e+00  3.4376833e-01]
 [ 3.2326165e-01  6.4587188e-01 -5.0333804e-01 ...  2.3226910e+00
   4.8779762e-01  2.5906691e-01]
 [ 6.3489598e-01 -7.0964795e-01 -6.4527944e-02 ...  3.7633318e-01
  -5.0703156e-01  2.7654967e-01]
 ...
 [ 1.2745025e-02 -1.6773384e-02 -1.4938287e-04 ... -1.3545465e-02
  -9.3904706e-03 -2.9300526e-02]
 [ 1.4688429e-02 -1.7455233e-02  3.8136836e-04 ... -5.4283887e-03
   8.9500155e-03 -2.3482623e-02]
 [ 3.0503512e-04 -1.5265445e-02  8.6061331e-03 ... -3.2664768e-02
  -4.0915638e-02 -2.3534561e-02]]


In [30]:

model_2.wv.most_similar(positive=["ohio"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('michigan', 0.7848071455955505),
 ('alabama', 0.7613809704780579),
 ('virginia', 0.7533582448959351),
 ('indiana', 0.7522750496864319),
 ('georgia', 0.7475205659866333),
 ('nj', 0.7438889741897583),
 ('minnesota', 0.7418457865715027),
 ('massachusetts', 0.7392698526382446),
 ('nc', 0.7315444946289062),
 ('maryland', 0.7303110361099243),
 ('texas', 0.7258538007736206),
 ('florida', 0.7253085970878601),
 ('utah', 0.7245694398880005),
 ('colorado', 0.7242205142974854),
 ('arizona', 0.7234482169151306),
 ('illinois', 0.7203905582427979),
 ('pennsylvania', 0.7190428376197815),
 ('missouri', 0.7177852392196655),
 ('ny', 0.7140328884124756),
 ('oregon', 0.7042700052261353)]

In [2]:
model_3=Word2Vec.load("cfpb_word2vec_gensim.model")

In [34]:
model_3.wv.most_similar(positive=["ohio","chicago"], negative=["columbus"])

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('minnesota', 0.576392412185669),
 ('alabama', 0.5629942417144775),
 ('michigan', 0.5611923933029175),
 ('illinois', 0.5545936822891235),
 ('virginia', 0.5493667125701904),
 ('utah', 0.5438264012336731),
 ('georgia', 0.5382406115531921),
 ('indiana', 0.5366226434707642),
 ('massachusetts', 0.5328266620635986),
 ('colorado', 0.5229722261428833)]